In [13]:
%pip install langchain faiss-cpu transformers torch accelerate sentence-transformers dashscope

In [14]:
%pip install dashscope langchain-community

In [15]:
!pip install pymupdf

In [16]:
# -*- coding: utf-8 -*-
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms import Tongyi  # 新版接口
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import fitz  # PyMuPDF
import pickle
import pandas as pd

# ================== 1. 加载并切分文档 ==================
def load_and_split_documents(file_path="/content/data/公平竞争审查条例.txt"):
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    return docs

# 1. 支持多种类型的文档加载，并统一封装为 LangChain Document 对象
def load_documents(file_path):
    documents = []

    if file_path.endswith('.pdf'):
        with fitz.open(file_path) as doc:
            for page_num, page in enumerate(doc):
                text = page.get_text()
                if text.strip():  # 避免空页
                    documents.append(
                        Document(page_content=text, metadata={"source": file_path, "page": page_num + 1})
                    )

    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
        for _, row in df.iterrows():
            content = ' '.join(str(value) for value in row.to_dict().values())
            documents.append(
                Document(page_content=content, metadata={"source": file_path})
            )

    elif file_path.endswith('.txt'):
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()  # 已经是 Document 类型列表
        documents.extend(docs)

    else:
        raise ValueError("Unsupported file type")

    return documents


# 2. 统一文本分割
def split_documents(documents):
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    split_docs = text_splitter.split_documents(documents)
    return split_docs

# ================== 2. 初始化 Embedding 模型 ==================
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

# ================== 3. 构建向量数据库 ==================
# def build_vectorstore(docs, embeddings):
#     vectorstore = FAISS.from_documents(docs, embeddings)
#     return vectorstore


def build_vectorstore(docs, embeddings):
    # 检查 docs 是否为空
    if not docs:
        raise ValueError("文档列表为空，无法构建向量数据库。请检查文档是否正确加载和分割。")

    # 可选：检查是否有空内容的文档
    valid_docs = [doc for doc in docs if doc.page_content.strip()]
    if len(valid_docs) == 0:
        raise ValueError("所有文档内容为空，无法构建向量数据库。请检查文档内容是否有效。")

    print(f"将使用 {len(valid_docs)} 个非空文档构建向量数据库")

    try:
        vectorstore = FAISS.from_documents(valid_docs, embeddings)
        return vectorstore
    except Exception as e:
        raise RuntimeError(f"构建向量数据库时出错：{e}")


#4. 长久保存向量数据库
def save_vectorstore(vectorstore, file_path="vectorstore.pkl"):
    with open(file_path, 'wb') as f:
        pickle.dump(vectorstore, f)

def load_vectorstore(file_path="vectorstore.pkl"):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

#5. 整合流程
def process_and_save_documents(file_path):
    documents = load_documents(file_path)
    #查看document里的内容
    for i, doc in enumerate(documents[:3], 1):
      print(f"Document {i}:")
      print("Content:")
      print(doc.page_content[:200] + "...")  # 限制打印长度，避免输出过多
      print("Metadata:", doc.metadata)
      print("-" * 50)
    docs = split_documents(documents)
    embeddings = get_embeddings()
    vectorstore = build_vectorstore(docs, embeddings)
    save_vectorstore(vectorstore)
    return vectorstore

# ================== 4. 初始化 Qwen LLM（Tongyi）==================
def get_llm(api_key):
    llm = Tongyi(model_name="qwen-max", dashscope_api_key=api_key)
    return llm

# ================== 5. 定义 Prompt 模板 ==================
def get_prompt_template():
    template = """用户问题：{question}

相关法律条文：{context}

请简要判断该政策是否违反公平竞争原则，并说明依据：
"""
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    return prompt

# ================== 6. 构建 QA 链 ==================
def build_qa_chain(vectorstore, llm, prompt):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# ================== 主函数入口 ==================
def main():
    # 设置 DashScope API Key
    DASHSCOPE_API_KEY = "sk-12cee5f31a6d4c0b9c000019cdeba574"  # 替换为你的实际 API Key
    os.environ["DASHSCOPE_API_KEY"] = DASHSCOPE_API_KEY

    # 步骤 1：加载并切分文档
    print("正在加载并切分文档...")
    docs = load_and_split_documents()

    # 步骤 2：初始化 Embedding 模型
    print("正在加载 Embedding 模型...")
    embeddings = get_embeddings()

    # 步骤 3：构建 FAISS 向量数据库
    print("正在构建向量数据库...")
    # vectorstore = build_vectorstore(docs, embeddings)
    file_path = "/content/公平竞争审查条例.txt"
    vectorstore = process_and_save_documents(file_path)

    # 步骤 4：初始化 LLM
    print("正在加载 Qwen 模型...")
    llm = get_llm(DASHSCOPE_API_KEY)

    # 步骤 5：定义 Prompt 模板
    print("正在构建 Prompt 模板...")
    prompt = get_prompt_template()

    # 步骤 6：构建 QA 链
    print("正在整合 RetrievalQA 链...")
    qa_chain = build_qa_chain(vectorstore, llm, prompt)

    # 示例查询
    query = "某地出台政策，要求外地企业必须在本地设立分支机构才能参与政府采购，是否合规？"

    print("\n开始推理...\n")
    result = qa_chain.run(query)

    print("回答结果：")
    print(result)

if __name__ == "__main__":
    main()

正在加载并切分文档...
正在加载 Embedding 模型...
正在构建向量数据库...
Document 1:
Content:
第一条为了规范公平竞争审查工作，促进市场公平竞争，优化营商环建设全国统一大市场，根据《中华人民共和国反垄断法》等法律，制定本境，条例。
第二条起草涉及经营者经济活动的法律、行政法规、地方性法规、规章、规范性文件以及具体政策措施(以下统称政策措施)，行政机关和法律法规授权的具有管理公共事务职能的组织(以下统称起草单位)应当依照本条例规定开展公平竞争审查。
第三条公平竞争审查工作坚持中国共产党的领导，...
Metadata: {'source': '/content/公平竞争审查条例.txt'}
--------------------------------------------------
将使用 1 个非空文档构建向量数据库
正在加载 Qwen 模型...
正在构建 Prompt 模板...
正在整合 RetrievalQA 链...

开始推理...



<ipython-input-16-664424280408>:179: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)


回答结果：
**判断：该政策违反公平竞争原则。**

**依据：**

1. **《中华人民共和国反垄断法》及公平竞争审查制度的相关规定**  
   根据相关法律条文，行政机关和法律法规授权的具有管理公共事务职能的组织在制定政策措施时，应当开展公平竞争审查（第二条）。政策要求外地企业必须在本地设立分支机构才能参与政府采购，这一规定可能构成对市场主体的不当限制，违背了公平竞争的原则。

2. **妨碍全国统一大市场的形成**  
   该政策与建设全国统一大市场的要求相悖（第一条）。外地企业因未在当地设立分支机构而被排除在政府采购之外，这种做法可能导致市场分割，阻碍生产要素的自由流动，损害其他地区经营者的合法权益。

3. **违反平等使用生产要素和公平参与市场竞争的原则**  
   根据第三条，国家保障各类经营者依法平等使用生产要素、公平参与市场竞争。该政策对外地企业附加额外条件（设立分支机构），显然对其参与政府采购设置了不合理门槛，构成了歧视性待遇。

4. **缺乏合理性与必要性**  
   政策未体现合理性和必要性，也未证明设立分支机构是参与政府采购的必需条件。这种做法可能只是为了保护本地企业的利益，而非基于公共服务或采购效率的实际需求，从而违反了公平竞争审查的要求。

综上所述，该政策不符合公平竞争审查制度的要求，涉嫌违反《中华人民共和国反垄断法》等相关法律法规，应予以修正或废止。
